In [ ]:
# NOTE: switch to the parent directory
%cd ..

In [ ]:
# Imports
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, confusion_matrix, roc_curve, precision_recall_curve,
                            auc, accuracy_score)
from sklearn.dummy import DummyClassifier

from copy import deepcopy

from utils.preproc_utils import run_preprocessing
from utils.plotting_utils import plotting_setup, CB_COLOR_CYCLE, STYLES

from tqdm import tqdm

In [ ]:
# Constants
TARGET_VARIABLE = 'DiagnosisByCriteria' # 'DiagnosisByCriteria', 'TreatmentGroupBinar', 'AppendicitisComplications'
SEED = 1799

In [ ]:
# Utility functions
def bootstrap_resample(data, labels):
    n_data_points = len(data)
    bootstrap_indices = np.random.choice(n_data_points, size=n_data_points, replace=True)
    return data.iloc[bootstrap_indices], labels.iloc[bootstrap_indices]


def eval_classification_metrics(probas, ys, n_thresh=100):
    res = {'sens': np.zeros((len(probas), n_thresh)), 'specs': np.zeros((len(probas), n_thresh)),
           'ppvs': np.zeros((len(probas), n_thresh)), 'npvs': np.zeros((len(probas), n_thresh))}
    probas_cat = np.concatenate([deepcopy(x) for x in probas], axis=0)
    probas_cat = np.sort(probas_cat)

    threshs = np.linspace(probas_cat[0], probas_cat[-1], n_thresh)

    for b in range(len(probas)):
        probas_b = probas[b]
        y_b = ys[b]

        for i in range(len(threshs)):
            y_test_pred = (probas_b >= threshs[i]) * 1.0
            cm = confusion_matrix(y_b, y_test_pred)
            sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0])
            specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
            ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1])
            npv = cm[0, 0] / (cm[0, 0] + cm[1, 0])

            res['sens'][b, i] = sensitivity
            res['specs'][b, i] = specificity
            res['ppvs'][b, i] = ppv
            res['npvs'][b, i] = npv

    return res, threshs

In [ ]:
# Load the raw data
app_data_regensburg = pd.read_csv('./data/app_data.csv')
app_data_dusseldorf = pd.read_excel('./data/app_data_ext.csv')

In [ ]:
# Preprocess and impute the data
app_data_regensburg, app_data_dusseldorf = run_preprocessing(app_data_regensburg, app_data_dusseldorf)

In [ ]:
# Construct targets and design matrices
y_regensburg = app_data_regensburg[TARGET_VARIABLE]
X_regensburg = app_data_regensburg.drop(['DiagnosisByCriteria', 'TreatmentGroupBinar', 'AppendicitisComplications'], axis=1)

y_dusseldorf = app_data_dusseldorf[TARGET_VARIABLE]
X_dusseldorf = app_data_dusseldorf.drop(['DiagnosisByCriteria', 'TreatmentGroupBinar', 'AppendicitisComplications'], axis=1)

In [ ]:
# Number of bootstrap resamples
B = 500

# Fix seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)

# Performance metrics
aurocs = {'lr': [], 'gb': [], 'rf': []}
auprs = {'lr': [], 'gb': [], 'rf': []}
sensitivities = {'lr': [], 'gb': [], 'rf': []}
specificities = {'lr': [], 'gb': [], 'rf': []}
accuracies = {'lr': [], 'gb': [], 'rf': []}
balanced_accuracies = {'lr': [], 'gb': [], 'rf': []}
ppvs = {'lr': [], 'gb': [], 'rf': []}
npvs = {'lr': [], 'gb': [], 'rf': []}
cms = {'lr': [], 'gb': [], 'rf': []}
probas = {'lr': [], 'gb': [], 'rf': []}
ys = {'lr': [], 'gb': [], 'rf': []}

for b in tqdm(np.arange(B)):
    # Make a bootstrap resample of the internal and external sets
    X_regensburg_b, y_regensburg_b = bootstrap_resample(X_regensburg, y_regensburg)
    X_dusseldorf_b, y_dusseldorf_b = bootstrap_resample(X_dusseldorf, y_dusseldorf)

    # Predictive models
    model_lr_b = LogisticRegression(max_iter=5000, penalty=None, random_state=SEED)
    model_gb_b = GradientBoostingClassifier(n_estimators=100, random_state=SEED)
    model_rf_b = RandomForestClassifier(n_estimators=1000, random_state=SEED)

    # Train the models
    model_lr_b.fit(X_regensburg_b, y_regensburg_b)
    model_gb_b.fit(X_regensburg_b, y_regensburg_b)
    model_rf_b.fit(X_regensburg_b, y_regensburg_b)

    models_b = {'lr': model_lr_b, 'gb': model_gb_b, 'rf': model_rf_b}

    for model_type in ['lr', 'gb', 'rf']:
        model = models_b[model_type]
        y_test_proba = model.predict_proba(X_dusseldorf_b)[:, 1]
        auroc = roc_auc_score(y_dusseldorf_b, y_test_proba)
        precision, recall, _ = precision_recall_curve(y_dusseldorf_b, y_test_proba)
        aupr = average_precision_score(y_dusseldorf_b, y_test_proba)
        y_test_pred = model.predict(X_dusseldorf_b)
        cm = confusion_matrix(y_dusseldorf_b, y_test_pred)
        sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0])
        specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
        accuracy = accuracy_score(y_dusseldorf_b, y_test_pred)
        balanced_accuracy = (sensitivity + specificity) / 2
        ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1])
        npv = cm[0, 0] / (cm[0, 0] + cm[1, 0])

        # Log the metrics
        aurocs[model_type].append(auroc)
        auprs[model_type].append(aupr)
        sensitivities[model_type].append(sensitivity)
        specificities[model_type].append(specificity)
        accuracies[model_type].append(accuracy)
        balanced_accuracies[model_type].append(balanced_accuracy)
        ppvs[model_type].append(ppv)
        npvs[model_type].append(npv)
        cms[model_type].append(cm)
        probas[model_type].append(y_test_proba)
        ys[model_type].append(y_dusseldorf_b)

In [ ]:
# Evaluate the coin flip
aurocs['rand'] = []
auprs['rand'] = []
sensitivities['rand'] = []
specificities['rand'] = []
accuracies['rand'] = []
balanced_accuracies['rand'] = []
ppvs['rand'] = []
npvs['rand'] = []
cms['rand'] = []

# Fix seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)

for b in tqdm(np.arange(10000)):
    # Make a bootstrap resample of the internal and external sets
    X_regensburg_b, y_regensburg_b = bootstrap_resample(X_regensburg, y_regensburg)
    X_dusseldorf_b, y_dusseldorf_b = bootstrap_resample(X_dusseldorf, y_dusseldorf)

    y_test_pred = np.random.permutation(deepcopy(y_dusseldorf))

    auroc = roc_auc_score(y_dusseldorf_b, y_test_pred)
    precision, recall, _ = precision_recall_curve(y_dusseldorf_b, y_test_pred)
    aupr = average_precision_score(y_dusseldorf_b, y_test_pred)
    cm = confusion_matrix(y_dusseldorf_b, y_test_pred)
    sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0] + 1E-13)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1] + 1E-13)
    accuracy = accuracy_score(y_dusseldorf_b, y_test_pred)
    balanced_accuracy = (sensitivity + specificity) / 2
    ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1] + 1E-13)
    npv = cm[0, 0] / (cm[0, 0] + cm[1, 0] + 1E-13)

    # Log the metrics
    aurocs['rand'].append(auroc)
    auprs['rand'].append(aupr)
    sensitivities['rand'].append(sensitivity)
    specificities['rand'].append(specificity)
    accuracies['rand'].append(accuracy)
    balanced_accuracies['rand'].append(balanced_accuracy)
    ppvs['rand'].append(ppv)
    npvs['rand'].append(npv)
    cms['rand'].append(cm)

In [ ]:
# Print summary statistics for AUROC and AUPR
for model_type in ['rand', 'lr', 'rf', 'gb']:
    print(model_type + ' & ' + str(np.round(np.mean(aurocs[model_type]), 2)) + '$\pm$' + str(np.round(np.std(aurocs[model_type]), 2)) +
         ' & ' + str(np.round(np.mean(auprs[model_type]), 2)) + '$\pm$' + str(np.round(np.std(auprs[model_type]), 2)))

In [ ]:
# Sensitivy, specificity, PPV, NPV
res, threshs = eval_classification_metrics(probas['rf'], ys['rf'], n_thresh=50)

In [ ]:
plotting_setup(16)

fig = plt.figure()
plt.plot(threshs, np.median(res['sens'], axis=0), c=CB_COLOR_CYCLE[0], linewidth=2, linestyle=STYLES[0], label='Sensitivity')
plt.fill_between(
    threshs, np.quantile(res['sens'], 0.25, axis=0), np.quantile(res['sens'], 0.75, axis=0), color=CB_COLOR_CYCLE[0], alpha=0.1)

plt.plot(threshs, np.median(res['specs'], axis=0), c=CB_COLOR_CYCLE[1], linewidth=2, linestyle=STYLES[1], label='Specificity')
plt.fill_between(
    threshs, np.quantile(res['specs'], 0.25, axis=0), np.quantile(res['specs'], 0.75, axis=0), color=CB_COLOR_CYCLE[1], alpha=0.1)

plt.plot(threshs, np.median(res['ppvs'], axis=0), c=CB_COLOR_CYCLE[2], linewidth=2, linestyle=STYLES[2], label='PPV')
plt.fill_between(
    threshs, np.quantile(res['ppvs'], 0.25, axis=0), np.quantile(res['ppvs'], 0.75, axis=0), color=CB_COLOR_CYCLE[2], alpha=0.1)

plt.plot(threshs, np.median(res['npvs'], axis=0), c=CB_COLOR_CYCLE[3], linewidth=2, linestyle=STYLES[3], label='NPV')
plt.fill_between(
    threshs, np.quantile(res['npvs'], 0.25, axis=0), np.quantile(res['npvs'], 0.75, axis=0), color=CB_COLOR_CYCLE[3], alpha=0.1)

plt.xlabel('Threshold Value')
# plt.title('Diagnosis', weight='bold')
# plt.ylabel('Performance')

plt.grid(visible=True, axis='y')

leg = plt.legend(loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.3), frameon=False)
plt.savefig('notebooks/plots/classification_metrics_' + TARGET_VARIABLE + '.pdf', format='pdf', bbox_inches='tight')